In [1]:
import numpy as np
import math
import random
import pandas as pd
import os
import matplotlib.pyplot as plt

import glob
import gc
from utils import *
from tqdm import tqdm
import pickle

from keras.optimizers import *
from keras.models import Model
from keras.layers import *
from keras.layers.core import *
from keras.layers.convolutional import *
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.
/home/whytin/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/whytin/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/whytin/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/whytin/.local/lib/python3.5/site-packages/tensorflow/python

In [2]:
#GTX1080TI 
#CUDA version == 9.0.176
#CUDNN version == 7501
#tensorflow-gpu == 1.80
#keras == 2.1.4

#cpu
#Keras==2.1.4
#tensorflow==1.14.0

# Initialize the setting

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
class Config():
    def __init__(self):
        self.frame_l = 32 # the length of frames
        self.joint_n = 22 # the number of joints
        self.joint_d = 3 # the dimension of joints
        self.clc_coarse = 14 # the number of coarse class
        self.clc_fine = 28 # the number of fine-grained class
        self.feat_d = 231
        self.filters = 8
        self.data_dir = '../data/SHREC/'
C = Config()

# Building the model

In [5]:
def poses_diff(x):
    H, W = x.get_shape()[1],x.get_shape()[2]
    x = tf.subtract(x[:,1:,...],x[:,:-1,...])
    x = tf.image.resize_nearest_neighbor(x,size=[H.value,W.value],align_corners=False) # should not alignment here
    return x

def pose_motion(P,frame_l):
    P_diff_slow = Lambda(lambda x: poses_diff(x))(P)
    P_diff_slow = Reshape((frame_l,-1))(P_diff_slow)
    P_fast = Lambda(lambda x: x[:,::2,...])(P)
    P_diff_fast = Lambda(lambda x: poses_diff(x))(P_fast)
    P_diff_fast = Reshape((int(frame_l/2),-1))(P_diff_fast)
   
    P_faster = Lambda(lambda x: x[:,::4,...])(P)
    P_diff_faster = Lambda(lambda x: poses_diff(x))(P_faster)
    P_diff_faster = Reshape((int(frame_l/4),-1))(P_diff_faster)

    return P_diff_slow,P_diff_fast,P_diff_faster
    
def c1D(x,filters,kernel):
    
    x = SeparableConv1D(filters, kernel_size=kernel,padding='same',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    return x

def block(x,filters):
    x = c1D(x,filters,3)
    return x

def build_FM(frame_l=32,joint_n=22,joint_d=2,feat_d=231,filters=16):   
    M = Input(shape=(frame_l,feat_d))
    P = Input(shape=(frame_l,joint_n,joint_d))
    
    diff_slow,diff_fast,diff_faster = pose_motion(P,frame_l)
    
    x = c1D(M,filters,3)
    x = SpatialDropout1D(0.1)(x)
    x = MaxPooling1D(2)(x)
    x = SpatialDropout1D(0.1)(x)
    
    x_d_slow = c1D(diff_slow,filters,3)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)
    x_d_slow = MaxPool1D(2)(x_d_slow)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)
        
    x_d_fast = c1D(diff_fast,filters,3) 
    x_d_fast = SpatialDropout1D(0.1)(x_d_fast)

    x_d_faster = c1D(diff_faster,filters,5) 
    x_d_faster = SpatialDropout1D(0.1)(x_d_faster)

    x_d_faster = UpSampling1D(2)(x_d_faster)
    x_d_faster = SpatialDropout1D(0.1)(x_d_faster)
    
    x = concatenate([x,x_d_slow,x_d_fast,x_d_faster])
    x = SpatialDropout1D(0.1)(x)
    
    x_shortcut = x
    
    x = block(x,filters*2)
    x = MaxPool1D(2)(x)
    x = SpatialDropout1D(0.1)(x)
    
    x = block(x,filters*4)
    x = MaxPool1D(2)(x)
    x = SpatialDropout1D(0.1)(x)

    x = block(x,filters*8)

    x_shortcut = SeparableConv1D(filters*8, kernel_size=3,padding='same',use_bias=False)(x_shortcut)
    x_shortcut = BatchNormalization()(x_shortcut)
    x_shortcut = LeakyReLU(alpha=0.2)(x_shortcut)
    
    x_shortcut = MaxPool1D(4)(x_shortcut)
    
    x = add([x,x_shortcut])
    
    return Model(inputs=[M,P],outputs=x)

In [6]:
def build_LDT_Net(frame_l=32,joint_n=22,joint_d=3,feat_d=231,clc_num=14,filters=16):
   
    M = Input(name='M', shape=(C.frame_l,C.feat_d))  
    P = Input(name='P', shape=(C.frame_l,C.joint_n,C.joint_d)) 
    FM = build_FM(C.frame_l,C.joint_n,C.joint_d,C.feat_d,C.filters)
    x = FM([M,P])
    
    x = Dropout(0.5)(x)
    x = GlobalAveragePooling1D()(x)

    x = Dense(clc_num, activation='softmax')(x)
    model = Model(inputs=[M,P],outputs=x)
    return model

In [7]:
LDT_Net = build_LDT_Net(C.frame_l,C.joint_n,C.joint_d,C.feat_d,C.clc_fine,C.filters)

In [8]:
LDT_Net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
M (InputLayer)                  (None, 32, 231)      0                                            
__________________________________________________________________________________________________
P (InputLayer)                  (None, 32, 22, 3)    0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 4, 64)        11139       M[0][0]                          
                                                                 P[0][0]                          
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4, 64)        0           model_1[1][0]                    
__________

## load data

In [9]:
Train = pickle.load(open(C.data_dir+"train.pkl", "rb"))
Test = pickle.load(open(C.data_dir+"test.pkl", "rb"))

## Training

In [10]:
X_0 = []
X_1 = []
Y = []
for i in tqdm(range(len(Train['pose']))): 
    p = np.copy(Train['pose'][i]).reshape([-1,22,3])
    p = zoom(p,target_l=C.frame_l,joints_num=C.joint_n,joints_dim=C.joint_d)
    p = normlize_range(p)
    
    label = np.zeros(C.clc_fine)
    label[Train['fine_label'][i]-1] = 1   

    M = get_CG(p,C)

    X_0.append(M)
    X_1.append(p)
    Y.append(label)

X_0 = np.stack(X_0)  
X_1 = np.stack(X_1) 
Y = np.stack(Y)

100%|██████████| 1960/1960 [00:19<00:00, 101.88it/s]


In [11]:
X_test_0 = []
X_test_1 = []
Y_test = []
for i in tqdm(range(len(Test['pose']))): 
    p = np.copy(Test['pose'][i]).reshape([-1,22,3])
    p = zoom(p,target_l=C.frame_l,joints_num=C.joint_n,joints_dim=C.joint_d)
    p = normlize_range(p)
    
    label = np.zeros(C.clc_fine)
    label[Test['fine_label'][i]-1] = 1   

    M = get_CG(p,C)

    X_test_0.append(M)
    X_test_1.append(p)
    Y_test.append(label)

X_test_0 = np.stack(X_test_0) 
X_test_1 = np.stack(X_test_1)  
Y_test = np.stack(Y_test)

100%|██████████| 840/840 [00:08<00:00, 103.44it/s]


In [ ]:
from keras.callbacks import ModelCheckpoint
import keras

lr = 1e-2
LDT_Net.compile(loss="categorical_crossentropy",optimizer=adam(lr),metrics=['accuracy'])
lrScheduler = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, cooldown=5, min_lr=1e-3)
history = LDT_Net.fit([X_0,X_1],Y,
                        batch_size=len(Y),
                        epochs=800,
                        verbose=True,
                        shuffle=True,
                        callbacks=[lrScheduler],
                        validation_data=([X_test_0,X_test_1],Y_test)      
                        )

lr = 1e-3
LDT_Net.compile(loss="categorical_crossentropy",optimizer=adam(lr),metrics=['accuracy'])

filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max')
callbacks_list = [checkpoint]
history = LDT_Net.fit([X_0,X_1],Y,
                        batch_size=len(Y),
                        epochs=1000,
                        verbose=True,
                        shuffle=True,
                        callbacks=callbacks_list,
                        validation_data=([X_test_0,X_test_1],Y_test)      
                        )

Train on 1960 samples, validate on 840 samples
Epoch 1/800
1960/1960 [==============================] - 5s 3ms/step - loss: 4.3831 - acc: 0.0383 - val_loss: 3.5343 - val_acc: 0.0726
Epoch 2/800
1960/1960 [==============================] - 0s 157us/step - loss: 3.7332 - acc: 0.0561 - val_loss: 3.3806 - val_acc: 0.0750
Epoch 3/800
1960/1960 [==============================] - 0s 148us/step - loss: 3.3693 - acc: 0.0755 - val_loss: 3.1066 - val_acc: 0.1036
Epoch 4/800
1960/1960 [==============================] - 0s 154us/step - loss: 3.1609 - acc: 0.1056 - val_loss: 2.9405 - val_acc: 0.1214
Epoch 5/800
1960/1960 [==============================] - 0s 145us/step - loss: 2.9850 - acc: 0.1495 - val_loss: 2.8206 - val_acc: 0.1536
Epoch 6/800
1960/1960 [==============================] - 0s 146us/step - loss: 2.9552 - acc: 0.1520 - val_loss: 2.7310 - val_acc: 0.1726
Epoch 7/800
1960/1960 [==============================] - 0s 146us/step - loss: 2.8179 - acc: 0.1929 - val_loss: 2.6395 - val_acc: 0.1

Epoch 61/800
1960/1960 [==============================] - 0s 149us/step - loss: 0.7213 - acc: 0.7699 - val_loss: 0.7320 - val_acc: 0.7690
Epoch 62/800
1960/1960 [==============================] - 0s 148us/step - loss: 0.7242 - acc: 0.7760 - val_loss: 0.9113 - val_acc: 0.6976
Epoch 63/800
1960/1960 [==============================] - 0s 152us/step - loss: 0.7244 - acc: 0.7709 - val_loss: 1.5352 - val_acc: 0.5179
Epoch 64/800
1960/1960 [==============================] - 0s 147us/step - loss: 0.7096 - acc: 0.7612 - val_loss: 2.0407 - val_acc: 0.4286
Epoch 65/800
1960/1960 [==============================] - 0s 145us/step - loss: 0.6848 - acc: 0.7862 - val_loss: 1.7054 - val_acc: 0.4905
Epoch 66/800
1960/1960 [==============================] - 0s 148us/step - loss: 0.6825 - acc: 0.7913 - val_loss: 1.1061 - val_acc: 0.6500
Epoch 67/800
1960/1960 [==============================] - 0s 145us/step - loss: 0.6710 - acc: 0.7929 - val_loss: 0.8551 - val_acc: 0.7143
Epoch 68/800
1960/1960 [==========

Epoch 121/800
1960/1960 [==============================] - 0s 149us/step - loss: 0.3653 - acc: 0.8878 - val_loss: 0.6207 - val_acc: 0.8238
Epoch 122/800
1960/1960 [==============================] - 0s 147us/step - loss: 0.3913 - acc: 0.8765 - val_loss: 0.5184 - val_acc: 0.8500
Epoch 123/800
1960/1960 [==============================] - 0s 149us/step - loss: 0.3813 - acc: 0.8816 - val_loss: 0.7291 - val_acc: 0.7595
Epoch 124/800
1960/1960 [==============================] - 0s 147us/step - loss: 0.3607 - acc: 0.8939 - val_loss: 0.9823 - val_acc: 0.6583
Epoch 125/800
1960/1960 [==============================] - 0s 145us/step - loss: 0.3788 - acc: 0.8719 - val_loss: 0.6366 - val_acc: 0.7917
Epoch 126/800
1960/1960 [==============================] - 0s 150us/step - loss: 0.3722 - acc: 0.8837 - val_loss: 0.5143 - val_acc: 0.8369
Epoch 127/800
1960/1960 [==============================] - 0s 145us/step - loss: 0.3712 - acc: 0.8918 - val_loss: 0.4965 - val_acc: 0.8488
Epoch 128/800
1960/1960 [==

Epoch 180/800
1960/1960 [==============================] - 0s 147us/step - loss: 0.3208 - acc: 0.9020 - val_loss: 0.5313 - val_acc: 0.8214
Epoch 181/800
1960/1960 [==============================] - 0s 149us/step - loss: 0.3054 - acc: 0.9138 - val_loss: 0.5276 - val_acc: 0.8214
Epoch 182/800
1960/1960 [==============================] - 0s 149us/step - loss: 0.3080 - acc: 0.9092 - val_loss: 0.5054 - val_acc: 0.8262
Epoch 183/800
1960/1960 [==============================] - 0s 145us/step - loss: 0.3145 - acc: 0.9005 - val_loss: 0.4919 - val_acc: 0.8262
Epoch 184/800
1960/1960 [==============================] - 0s 145us/step - loss: 0.3091 - acc: 0.9087 - val_loss: 0.4865 - val_acc: 0.8333
Epoch 185/800
1960/1960 [==============================] - 0s 146us/step - loss: 0.3011 - acc: 0.9061 - val_loss: 0.4725 - val_acc: 0.8393
Epoch 186/800
1960/1960 [==============================] - 0s 146us/step - loss: 0.3035 - acc: 0.9128 - val_loss: 0.4523 - val_acc: 0.8524
Epoch 187/800
1960/1960 [==

Epoch 239/800
1960/1960 [==============================] - 0s 149us/step - loss: 0.3044 - acc: 0.8974 - val_loss: 0.4063 - val_acc: 0.8833
Epoch 240/800
1960/1960 [==============================] - 0s 150us/step - loss: 0.3062 - acc: 0.9026 - val_loss: 0.4247 - val_acc: 0.8643
Epoch 241/800
1960/1960 [==============================] - 0s 150us/step - loss: 0.2838 - acc: 0.9214 - val_loss: 0.4610 - val_acc: 0.8464
Epoch 242/800
1960/1960 [==============================] - 0s 148us/step - loss: 0.2697 - acc: 0.9168 - val_loss: 0.4992 - val_acc: 0.8321
Epoch 243/800
1960/1960 [==============================] - 0s 148us/step - loss: 0.2945 - acc: 0.9077 - val_loss: 0.5197 - val_acc: 0.8226
Epoch 244/800
1960/1960 [==============================] - 0s 146us/step - loss: 0.2815 - acc: 0.9194 - val_loss: 0.5102 - val_acc: 0.8250
Epoch 245/800
1960/1960 [==============================] - 0s 147us/step - loss: 0.2805 - acc: 0.9138 - val_loss: 0.4726 - val_acc: 0.8405
Epoch 246/800
1960/1960 [==

...


Epoch 00719: val_acc did not improve
Epoch 720/1000
1960/1960 [==============================] - 0s 138us/step - loss: 0.1157 - acc: 0.9673 - val_loss: 0.4191 - val_acc: 0.9095

Epoch 00720: val_acc did not improve
Epoch 721/1000
1960/1960 [==============================] - 0s 134us/step - loss: 0.1395 - acc: 0.9551 - val_loss: 0.4267 - val_acc: 0.9119

Epoch 00721: val_acc did not improve
Epoch 722/1000
1960/1960 [==============================] - 0s 140us/step - loss: 0.1196 - acc: 0.9622 - val_loss: 0.4368 - val_acc: 0.9095

Epoch 00722: val_acc did not improve
Epoch 723/1000
1960/1960 [==============================] - 0s 142us/step - loss: 0.1383 - acc: 0.9541 - val_loss: 0.4710 - val_acc: 0.9071

Epoch 00723: val_acc did not improve
Epoch 724/1000
1960/1960 [==============================] - 0s 141us/step - loss: 0.1283 - acc: 0.9602 - val_loss: 0.5007 - val_acc: 0.9000

Epoch 00724: val_acc did not improve
Epoch 725/1000
1960/1960 [==============================] - 0s 138us/ste

1960/1960 [==============================] - 0s 138us/step - loss: 0.1162 - acc: 0.9592 - val_loss: 0.4619 - val_acc: 0.9060

Epoch 00766: val_acc did not improve
Epoch 767/1000
1960/1960 [==============================] - 0s 137us/step - loss: 0.1264 - acc: 0.9582 - val_loss: 0.4563 - val_acc: 0.9000

Epoch 00767: val_acc did not improve
Epoch 768/1000
1960/1960 [==============================] - 0s 125us/step - loss: 0.1225 - acc: 0.9597 - val_loss: 0.4683 - val_acc: 0.8929

Epoch 00768: val_acc did not improve
Epoch 769/1000
1960/1960 [==============================] - 0s 126us/step - loss: 0.1345 - acc: 0.9536 - val_loss: 0.4825 - val_acc: 0.8810

Epoch 00769: val_acc did not improve
Epoch 770/1000
1960/1960 [==============================] - 0s 140us/step - loss: 0.1313 - acc: 0.9597 - val_loss: 0.4548 - val_acc: 0.8881

Epoch 00770: val_acc did not improve
Epoch 771/1000
1960/1960 [==============================] - 0s 127us/step - loss: 0.1274 - acc: 0.9633 - val_loss: 0.4241 - v


Epoch 00812: val_acc did not improve
Epoch 813/1000
1960/1960 [==============================] - 0s 137us/step - loss: 0.1194 - acc: 0.9628 - val_loss: 0.4582 - val_acc: 0.9012

Epoch 00813: val_acc did not improve
Epoch 814/1000
1960/1960 [==============================] - 0s 137us/step - loss: 0.1216 - acc: 0.9592 - val_loss: 0.4215 - val_acc: 0.9036

Epoch 00814: val_acc did not improve
Epoch 815/1000
1960/1960 [==============================] - 0s 141us/step - loss: 0.1204 - acc: 0.9648 - val_loss: 0.4140 - val_acc: 0.9024

Epoch 00815: val_acc did not improve
Epoch 816/1000
1960/1960 [==============================] - 0s 140us/step - loss: 0.1333 - acc: 0.9587 - val_loss: 0.4243 - val_acc: 0.9024

Epoch 00816: val_acc did not improve
Epoch 817/1000
1960/1960 [==============================] - 0s 139us/step - loss: 0.1142 - acc: 0.9622 - val_loss: 0.4362 - val_acc: 0.9024

Epoch 00817: val_acc did not improve
Epoch 818/1000
1960/1960 [==============================] - 0s 139us/ste

1960/1960 [==============================] - 0s 138us/step - loss: 0.1311 - acc: 0.9561 - val_loss: 0.4137 - val_acc: 0.8940

Epoch 00859: val_acc did not improve
Epoch 860/1000
1960/1960 [==============================] - 0s 143us/step - loss: 0.1133 - acc: 0.9622 - val_loss: 0.4086 - val_acc: 0.9036

Epoch 00860: val_acc did not improve
Epoch 861/1000
1960/1960 [==============================] - 0s 137us/step - loss: 0.1128 - acc: 0.9648 - val_loss: 0.4202 - val_acc: 0.9048

Epoch 00861: val_acc did not improve
Epoch 862/1000
1960/1960 [==============================] - 0s 141us/step - loss: 0.1327 - acc: 0.9546 - val_loss: 0.4549 - val_acc: 0.9060

Epoch 00862: val_acc did not improve
Epoch 863/1000
1960/1960 [==============================] - 0s 139us/step - loss: 0.1375 - acc: 0.9546 - val_loss: 0.4854 - val_acc: 0.9048

Epoch 00863: val_acc did not improve
Epoch 864/1000
1960/1960 [==============================] - 0s 142us/step - loss: 0.1386 - acc: 0.9510 - val_loss: 0.5149 - v


Epoch 00905: val_acc did not improve
Epoch 906/1000
1960/1960 [==============================] - 0s 140us/step - loss: 0.1325 - acc: 0.9607 - val_loss: 0.4267 - val_acc: 0.9048

Epoch 00906: val_acc did not improve
Epoch 907/1000
1960/1960 [==============================] - 0s 139us/step - loss: 0.1293 - acc: 0.9582 - val_loss: 0.4298 - val_acc: 0.9024

Epoch 00907: val_acc did not improve
Epoch 908/1000
1960/1960 [==============================] - 0s 141us/step - loss: 0.1354 - acc: 0.9633 - val_loss: 0.4451 - val_acc: 0.8940

Epoch 00908: val_acc did not improve
Epoch 909/1000
1960/1960 [==============================] - 0s 137us/step - loss: 0.1191 - acc: 0.9694 - val_loss: 0.4558 - val_acc: 0.8952

Epoch 00909: val_acc did not improve
Epoch 910/1000
1960/1960 [==============================] - 0s 139us/step - loss: 0.1264 - acc: 0.9566 - val_loss: 0.4419 - val_acc: 0.8964

Epoch 00910: val_acc did not improve
Epoch 911/1000
1960/1960 [==============================] - 0s 140us/ste

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
LDT_Net.save_weights('weights/fine_lite.h5')